# E-Commerce Competitive Intelligence - Exploratory Data Analysis

**Author:** Jack Massey  
**Date:** 2026-01-25  
**Purpose:** Comprehensive EDA of 374 days of hourly web scraping data from a beauty/haircare e-commerce reseller

---

## Dataset Overview
- **Rows:** 1,724,927 observations
- **Time Period:** December 9, 2024 → December 19, 2025 (374 days)
- **Brands:** 24 beauty/haircare brands
- **Products:** 239 unique products with 280 variants
- **Frequency:** Hourly snapshots (8,947 timestamps)

This notebook consolidates the exploratory data analysis from scripts 01-03, providing interactive visualizations and insights into the competitive landscape.

---

## 1. Setup & Data Loading

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# Configure pandas display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 80)

print("✓ Libraries imported successfully")

In [ ]:
# Load cleaned data
DATA_PATH = Path('../data/processed/cleaned_data.parquet')

print(f"Loading data from: {DATA_PATH}")
df = pd.read_parquet(DATA_PATH)

print(f"\n✓ Loaded {len(df):,} rows × {len(df.columns)} columns")
print(f"✓ Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

In [ ]:
# Dataset overview
print("=" * 80)
print("DATASET OVERVIEW")
print("=" * 80)

print(f"\nShape: {df.shape}")
print(f"\nDate Range: {df['date'].min()} → {df['date'].max()}")
print(f"Duration: {(df['date'].max() - df['date'].min()).days} days")

print(f"\nUnique Counts:")
print(f"  - Brands: {df['brand'].nunique()}")
print(f"  - Products: {df['product_id'].nunique()}")
print(f"  - Variants: {df['offer_offer_id'].nunique()}")
print(f"  - Timestamps: {df['date'].nunique():,}")

print(f"\nData Types:")
print(df.dtypes)

In [ ]:
# Display basic info
df.info()

---

## 2. Initial Exploration (Column Analysis)

Understanding each column's characteristics: data type, unique values, missing data, and sample values.

In [ ]:
# Column-level analysis
def analyze_columns(df):
    """
    Analyze each column to understand:
    - Data type
    - Number of unique values
    - Number of missing values
    - Sample values
    """
    analysis = []
    
    for col in df.columns:
        n_unique = df[col].nunique()
        n_missing = df[col].isna().sum()
        pct_missing = (n_missing / len(df)) * 100
        dtype = df[col].dtype
        
        # Get sample values (first 3 non-null)
        sample_values = df[col].dropna().head(3).tolist()
        
        analysis.append({
            'column': col,
            'dtype': str(dtype),
            'n_unique': n_unique,
            'n_missing': n_missing,
            'pct_missing': round(pct_missing, 2),
            'sample_values': str(sample_values)[:100]  # Truncate for display
        })
    
    return pd.DataFrame(analysis)

# Run analysis
column_analysis = analyze_columns(df)

print("=" * 100)
print("COLUMN ANALYSIS")
print("=" * 100)
column_analysis

In [ ]:
# Visualize missing data
missing_data = df.isnull().sum()
missing_data = missing_data[missing_data > 0].sort_values(ascending=False)

if len(missing_data) > 0:
    fig, ax = plt.subplots(figsize=(10, 4))
    missing_pct = (missing_data / len(df)) * 100
    missing_pct.plot(kind='barh', ax=ax, color='coral')
    ax.set_xlabel('Missing Data (%)')
    ax.set_title('Missing Data by Column')
    ax.grid(axis='x', alpha=0.3)
    
    for i, v in enumerate(missing_pct):
        ax.text(v + 0.1, i, f'{v:.2f}%', va='center')
    
    plt.tight_layout()
    plt.show()
else:
    print("✓ No missing values detected (except description field)")

### Problematic Columns Identification

Looking for columns that might be problematic:
- Constant columns (only 1 unique value)
- Empty columns (all nulls)
- High missing rate (>50%)

In [ ]:
def identify_problematic_columns(df, analysis_df):
    """
    Identify columns that might be useless or problematic.
    """
    issues = []
    
    for idx, row in analysis_df.iterrows():
        col = row['column']
        
        # Constant columns
        if row['n_unique'] == 1:
            issues.append({
                'column': col,
                'issue': 'CONSTANT',
                'detail': f"Only 1 unique value",
                'recommendation': 'REMOVE - No variance'
            })
        
        # All missing
        elif row['pct_missing'] == 100:
            issues.append({
                'column': col,
                'issue': 'EMPTY',
                'detail': 'All values are null',
                'recommendation': 'REMOVE - No data'
            })
        
        # High missing rate (>50%)
        elif row['pct_missing'] > 50:
            issues.append({
                'column': col,
                'issue': 'HIGH_MISSING',
                'detail': f"{row['pct_missing']}% missing",
                'recommendation': 'INVESTIGATE - Might be optional field'
            })
    
    return pd.DataFrame(issues) if issues else None

# Identify issues
issues_df = identify_problematic_columns(df, column_analysis)

if issues_df is not None and len(issues_df) > 0:
    print("=" * 100)
    print("PROBLEMATIC COLUMNS IDENTIFIED")
    print("=" * 100)
    display(issues_df)
else:
    print("✓ No obvious problematic columns found in cleaned dataset")

### Outlier Detection (Numeric Columns)

Detecting potential data errors using the 3-sigma rule (values >3 standard deviations from mean).

In [ ]:
def detect_potential_errors(df):
    """
    Detect potential data entry errors in numeric columns.
    Look for outliers (>3 standard deviations from mean).
    """
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    outliers_summary = []
    
    for col in numeric_cols:
        mean = df[col].mean()
        std = df[col].std()
        
        if std == 0:  # Skip columns with no variance
            continue
        
        # Find outliers (>3 std from mean)
        outliers = df[(df[col] > mean + 3*std) | (df[col] < mean - 3*std)]
        
        if len(outliers) > 0:
            outliers_summary.append({
                'column': col,
                'n_outliers': len(outliers),
                'pct_outliers': round((len(outliers) / len(df)) * 100, 2),
                'mean': round(mean, 2),
                'std': round(std, 2),
                'outlier_range': f"<{round(mean - 3*std, 2)} or >{round(mean + 3*std, 2)}"
            })
    
    return pd.DataFrame(outliers_summary) if outliers_summary else None

# Detect outliers
outliers_df = detect_potential_errors(df)

if outliers_df is not None:
    print("=" * 100)
    print("POTENTIAL DATA ERRORS (Outliers)")
    print("=" * 100)
    display(outliers_df)
else:
    print("✓ No significant outliers detected in numeric columns")

In [ ]:
# Visualize distributions of numeric columns
numeric_cols = ['offer_price', 'stock']

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Price distribution
axes[0].hist(df['offer_price'], bins=50, color='skyblue', edgecolor='black', alpha=0.7)
axes[0].axvline(df['offer_price'].mean(), color='red', linestyle='--', linewidth=2, label=f"Mean: €{df['offer_price'].mean():.2f}")
axes[0].axvline(df['offer_price'].median(), color='green', linestyle='--', linewidth=2, label=f"Median: €{df['offer_price'].median():.2f}")
axes[0].set_xlabel('Price (EUR)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Price Distribution')
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)

# Stock distribution
axes[1].hist(df['stock'], bins=50, color='lightcoral', edgecolor='black', alpha=0.7)
axes[1].axvline(df['stock'].mean(), color='red', linestyle='--', linewidth=2, label=f"Mean: {df['stock'].mean():.1f} units")
axes[1].axvline(df['stock'].median(), color='green', linestyle='--', linewidth=2, label=f"Median: {df['stock'].median():.1f} units")
axes[1].set_xlabel('Stock (units)')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Stock Distribution')
axes[1].legend()
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nPrice Stats: Min=€{df['offer_price'].min():.2f}, Max=€{df['offer_price'].max():.2f}, Mean=€{df['offer_price'].mean():.2f}")
print(f"Stock Stats: Min={df['stock'].min()}, Max={df['stock'].max()}, Mean={df['stock'].mean():.1f}")

---

## 3. Data Structure Understanding

Deep dive into what each column represents and how data is structured.

In [ ]:
# Display first 10 rows
print("=" * 100)
print("FIRST 10 ROWS - Understanding Data Structure")
print("=" * 100)

df.head(10)

### Column-by-Column Interpretation

In [ ]:
print("=" * 100)
print("COLUMN-BY-COLUMN EXAMINATION")
print("=" * 100)

# 1. Product ID
print("\n1. PRODUCT_ID - URL slug identifier")
print("-" * 100)
print(f"   Total unique: {df['product_id'].nunique()}")
print(f"   Sample values:")
for val in df['product_id'].head(5):
    print(f"      • {val}")
print(f"   Pattern: URL-friendly slugs (lowercase, hyphenated product names)")

# 2. Title
print("\n2. TITLE - Product names")
print("-" * 100)
print(f"   Total unique: {df['title'].nunique()}")
print(f"   Sample values:")
for val in df['title'].head(5):
    print(f"      • {val}")
print(f"   Pattern: Human-readable product names (French, uppercase)")

# 3. Description
print("\n3. DESCRIPTION - Product details")
print("-" * 100)
print(f"   Total unique: {df['description'].nunique()}")
print(f"   Missing: {df['description'].isna().sum()} ({df['description'].isna().sum()/len(df)*100:.2f}%)")
if df['description'].notna().sum() > 0:
    first_desc = df['description'].dropna().iloc[0]
    print(f"   Sample value (first non-null):")
    print(f"      {first_desc[:200]}...")

# 4. Brand
print("\n4. BRAND - All brands")
print("-" * 100)
brands = sorted(df['brand'].unique())
print(f"   Total unique: {len(brands)}")
print(f"   All brands:")
for i, brand in enumerate(brands, 1):
    print(f"      {i:2d}. {brand}")

# 5. Offer Title
print("\n5. OFFER_TITLE - Product variants")
print("-" * 100)
print(f"   Total unique: {df['offer_title'].nunique()}")
print(f"   Most common values:")
top_offers = df['offer_title'].value_counts().head(10)
for offer, count in top_offers.items():
    print(f"      • '{offer}': {count:,} occurrences")

# 6. Offer Offer ID
print("\n6. OFFER_OFFER_ID - Unique variant identifier")
print("-" * 100)
print(f"   Total unique: {df['offer_offer_id'].nunique()}")
print(f"   Sample values: {df['offer_offer_id'].head(5).tolist()}")
print(f"   Pattern: Shopify variant ID (13-digit integer)")

# 7. Offer Price
print("\n7. OFFER_PRICE - Price in EUR")
print("-" * 100)
print(f"   Range: €{df['offer_price'].min():.2f} - €{df['offer_price'].max():.2f}")
print(f"   Mean: €{df['offer_price'].mean():.2f}")
print(f"   Median: €{df['offer_price'].median():.2f}")

# 8. Offer In Stock
print("\n8. OFFER_IN_STOCK - Boolean availability flag")
print("-" * 100)
stock_status = df['offer_in_stock'].value_counts()
for status, count in stock_status.items():
    pct = count / len(df) * 100
    print(f"      • {status}: {count:,} ({pct:.2f}%)")

# 9. Stock
print("\n9. STOCK - Quantity available")
print("-" * 100)
print(f"   Range: {df['stock'].min()} - {df['stock'].max()} units")
print(f"   Mean: {df['stock'].mean():.2f} units")
print(f"   Median: {df['stock'].median():.2f} units")

# 10. Date
print("\n10. DATE - Timestamp of scrape")
print("-" * 100)
print(f"   Data type: {df['date'].dtype}")
print(f"   Unique timestamps: {df['date'].nunique():,}")
print(f"   First timestamp: {df['date'].min()}")
print(f"   Last timestamp: {df['date'].max()}")

### Data Relationships & Hierarchy

In [ ]:
print("=" * 100)
print("DATA RELATIONSHIPS & HIERARCHY")
print("=" * 100)

# Products per brand
print("\n1. How many products does each brand have?")
print("-" * 100)
products_per_brand = df.groupby('brand')['product_id'].nunique().sort_values(ascending=False)
for brand, count in products_per_brand.items():
    print(f"   {brand:30s}: {count:3d} products")

In [ ]:
# Visualize products per brand
fig = px.bar(products_per_brand.reset_index(), 
             x='brand', 
             y='product_id',
             title='Number of Products per Brand',
             labels={'product_id': 'Number of Products', 'brand': 'Brand'},
             color='product_id',
             color_continuous_scale='Blues')
fig.update_layout(xaxis_tickangle=-45, height=500)
fig.show()

In [ ]:
# Offers per product (variants)
print("\n2. Do products have multiple variants (offers)?")
print("-" * 100)
offers_per_product = df.groupby('product_id')['offer_offer_id'].nunique()
print(f"   Products with 1 variant: {(offers_per_product == 1).sum()}")
print(f"   Products with 2+ variants: {(offers_per_product > 1).sum()}")
print(f"   Max variants per product: {offers_per_product.max()}")
print(f"\n   Example - Products with multiple variants:")
multi_variant_products = offers_per_product[offers_per_product > 1].head(10)
for product, n_variants in multi_variant_products.items():
    product_title = df[df['product_id'] == product]['title'].iloc[0]
    print(f"      • '{product_title}': {n_variants} variants")

In [ ]:
# Temporal coverage per brand
print("\n3. Do all brands have the same temporal coverage?")
print("-" * 100)
brand_dates = df.groupby('brand')['date'].agg(['min', 'max', 'nunique']).reset_index()
brand_dates.columns = ['Brand', 'First_Date', 'Last_Date', 'N_Timestamps']
brand_dates = brand_dates.sort_values('N_Timestamps', ascending=False)
brand_dates

In [ ]:
# Visualize temporal coverage
fig = px.bar(brand_dates, 
             x='Brand', 
             y='N_Timestamps',
             title='Temporal Coverage by Brand (Number of Timestamps)',
             labels={'N_Timestamps': 'Number of Timestamps', 'Brand': 'Brand'},
             color='N_Timestamps',
             color_continuous_scale='Viridis')
fig.update_layout(xaxis_tickangle=-45, height=500)
fig.show()

print(f"\n📊 Insight: {(brand_dates['N_Timestamps'] == brand_dates['N_Timestamps'].max()).sum()} brands have complete coverage")

---

## 4. Detailed Analysis

Deep dive into specific aspects: brands, pricing, stock levels, and variants.

### Brand Analysis

In [ ]:
print("=" * 120)
print("BRAND ANALYSIS")
print("=" * 120)

brands = df['brand'].unique()
print(f"\nTotal unique brand names: {len(brands)}\n")

print("All brand names with statistics:")
print("-" * 120)
brand_stats = []
for brand in sorted(brands):
    count = (df['brand'] == brand).sum()
    n_products = df[df['brand'] == brand]['product_id'].nunique()
    brand_stats.append({
        'Brand': brand,
        'Total_Rows': count,
        'N_Products': n_products
    })

brand_stats_df = pd.DataFrame(brand_stats)
brand_stats_df

In [ ]:
# Brand duplicate investigation
print("\n=" * 120)
print("BRAND DUPLICATE INVESTIGATION")
print("=" * 120)

# Look for similar brand names
potential_duplicates = []

for i, brand1 in enumerate(brands):
    for brand2 in brands[i+1:]:
        # Check for similarity (case-insensitive, whitespace-normalized)
        norm1 = brand1.strip().upper().replace(' ', '')
        norm2 = brand2.strip().upper().replace(' ', '')
        
        if norm1 == norm2:
            potential_duplicates.append((brand1, brand2))

if potential_duplicates:
    print("\n DUPLICATE BRANDS DETECTED:")
    for brand1, brand2 in potential_duplicates:
        count1 = (df['brand'] == brand1).sum()
        count2 = (df['brand'] == brand2).sum()
        products1 = df[df['brand'] == brand1]['product_id'].nunique()
        products2 = df[df['brand'] == brand2]['product_id'].nunique()
        
        print(f"\n   Brand 1: '{brand1}' ({count1:,} rows, {products1} products)")
        print(f"   Brand 2: '{brand2}' ({count2:,} rows, {products2} products)")
        print(f"   RECOMMENDATION: These should be merged under a single name")
else:
    print("\n✓ No obvious duplicates found based on normalized names")
    print("  (Note: Data cleaning script already merged 'AUXPORTESDUNATUREL' duplicates)")

In [ ]:
# Product-brand consistency check
print("\n=" * 120)
print("PRODUCT-BRAND CONSISTENCY CHECK")
print("=" * 120)

# Count brands per product
product_brands = df.groupby('product_id')['brand'].nunique()
multi_brand_products = product_brands[product_brands > 1]

if len(multi_brand_products) > 0:
    print(f"\n   ⚠️  ISSUE FOUND: {len(multi_brand_products)} products appear under multiple brands!")
    print(f"\n   Examples:")
    for product in multi_brand_products.head(5).index:
        brands_list = df[df['product_id'] == product]['brand'].unique()
        print(f"      • Product '{product}' appears under: {', '.join(brands_list)}")
    print(f"\n   RECOMMENDATION: Investigate and standardize brand assignment")
else:
    print("\n   ✓ All products consistently assigned to single brand")

### Price Distribution Analysis

In [ ]:
# Price distribution by brand
fig = px.box(df, 
             x='brand', 
             y='offer_price',
             title='Price Distribution by Brand',
             labels={'offer_price': 'Price (EUR)', 'brand': 'Brand'},
             color='brand')
fig.update_layout(xaxis_tickangle=-45, height=600, showlegend=False)
fig.show()

In [ ]:
# Price ranges
print("\nPRICE DISTRIBUTION BY RANGE")
print("-" * 80)
price_ranges = [0, 10, 20, 30, 50, 100, 200]
for i in range(len(price_ranges)-1):
    count = ((df['offer_price'] >= price_ranges[i]) & (df['offer_price'] < price_ranges[i+1])).sum()
    pct = count / len(df) * 100
    print(f"   €{price_ranges[i]:3d}-€{price_ranges[i+1]:3d}: {count:8,} ({pct:5.2f}%)")

# Over max range
count_over = (df['offer_price'] >= price_ranges[-1]).sum()
pct_over = count_over / len(df) * 100
print(f"   €{price_ranges[-1]:3d}+     : {count_over:8,} ({pct_over:5.2f}%)")

### Stock Distribution Analysis

In [ ]:
# Stock distribution by brand
fig = px.box(df, 
             x='brand', 
             y='stock',
             title='Stock Distribution by Brand',
             labels={'stock': 'Stock (units)', 'brand': 'Brand'},
             color='brand')
fig.update_layout(xaxis_tickangle=-45, height=600, showlegend=False)
fig.show()

In [ ]:
# Stock ranges
print("\nSTOCK DISTRIBUTION BY RANGE")
print("-" * 80)
stock_ranges = [0, 10, 25, 50, 100, 200, 800]
for i in range(len(stock_ranges)-1):
    count = ((df['stock'] >= stock_ranges[i]) & (df['stock'] < stock_ranges[i+1])).sum()
    pct = count / len(df) * 100
    print(f"   {stock_ranges[i]:3d}-{stock_ranges[i+1]:3d} units: {count:8,} ({pct:5.2f}%)")

# Over max range
count_over = (df['stock'] >= stock_ranges[-1]).sum()
pct_over = count_over / len(df) * 100
print(f"   {stock_ranges[-1]:3d}+ units     : {count_over:8,} ({pct_over:5.2f}%)")

### Variant Analysis

In [ ]:
# Offer title distribution
print("\nOFFER VARIANT ANALYSIS")
print("-" * 80)

offer_counts = df['offer_title'].value_counts()
print(f"\nTotal unique offer variants: {len(offer_counts)}")
print(f"\nTop 15 most common variants:")
for i, (offer, count) in enumerate(offer_counts.head(15).items(), 1):
    pct = (count / len(df)) * 100
    print(f"   {i:2d}. '{offer}': {count:8,} ({pct:5.2f}%)")

# Count default vs custom variants
default_count = (df['offer_title'] == 'Default Title').sum()
custom_count = len(df) - default_count
print(f"\nVariant Type Breakdown:")
print(f"   Default Title: {default_count:,} ({default_count/len(df)*100:.2f}%)")
print(f"   Custom Variants: {custom_count:,} ({custom_count/len(df)*100:.2f}%)")

In [ ]:
# Visualize variant distribution
variant_data = pd.DataFrame({
    'Variant Type': ['Default Title', 'Custom Variants'],
    'Count': [default_count, custom_count]
})

fig = px.pie(variant_data, 
             values='Count', 
             names='Variant Type',
             title='Product Variants: Default vs Custom',
             color_discrete_sequence=['#636EFA', '#EF553B'])
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

### Date Format Consistency

In [ ]:
print("\nDATE FORMAT CONSISTENCY CHECK")
print("-" * 80)

# Sample dates throughout dataset
sample_indices = np.linspace(0, len(df)-1, 10, dtype=int)
sample_dates = df.iloc[sample_indices]['date']

print("\n   Sample dates from throughout dataset:")
for i, date in enumerate(sample_dates, 1):
    print(f"      {i}. {date}")

print(f"\n   ✓ Data type: {df['date'].dtype} (datetime64)")
print(f"   ✓ Format is consistent and properly converted")

---

## 5. Insights Summary

Key findings from the exploratory data analysis.

### Key Findings

#### Data Quality
- **Clean Dataset**: All problematic columns (constant values, empty fields, URLs) have been removed in the cleaning process
- **Missing Data**: Only 5.66% of descriptions are missing, which is acceptable for an optional field
- **Brand Consistency**: All products are consistently assigned to a single brand (no conflicts)
- **Date Format**: All timestamps properly converted to datetime64 format

#### Brand Portfolio
- **24 unique brands** in the dataset (down from 25 after merging duplicates)
- **Uneven distribution**: Top brands like LES SECRETS DE LOLY (27 products), CUT BY FRED (23), and CENTIFOLIA (20)
- **Temporal coverage**: Most brands have complete coverage across all 8,947 timestamps, indicating stable tracking

#### Product & Variants
- **239 unique products** with **280 variants** total
- **218 products** have only 1 variant ("Default Title")
- **21 products** have multiple variants (colors, sizes, subscription periods)
- **Maximum 11 variants** per product (likely gift cards or subscription products)
- **89% of records** use "Default Title" (single-variant products dominate)

#### Pricing Insights
- **Price range**: €0 - €180, with mean of €22.12 and median slightly lower
- **Most products (47%)** priced in the €10-€20 range (accessible price point)
- **Premium segment**: Only 3.37% of products above €50
- **Price distribution** varies significantly by brand, reflecting different positioning strategies

#### Inventory Insights
- **Stock availability**: 96.17% of records show products in stock (healthy inventory)
- **Stock levels**: Range from 0 to 741 units, mean of 41 units
- **Low stockout rate**: Only 3.83% out of stock, indicating good inventory management
- **Stock distribution** shows most products maintain moderate inventory (25-100 units)

#### Potential Data Quality Issues
- **Price outliers**: 1.84% of records show unusual pricing (likely data entry errors or special promotions)
- **Stock outliers**: 1.96% of records with unusual stock levels (possibly bulk restocks or errors)
- These outliers are minimal and don't significantly impact overall analysis

The cleaned dataset is ready for advanced analytics and visualization in the Streamlit dashboard.